In [1]:
import pandas as pd
import requests
import os
from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

pd.set_option('display.max_columns',30)

In [2]:
"""
Steps to do

1. Get the data from S3
2. weather data transformations
3. Do the transformation like in 06 notebooks
4. Update payment type
5. Update company
6. update taxi trips with company and paymant types (replace str values with ids from the latest master tables)
7. upload the weather to S3
8. upload the taxi data to S3
9. upload the newest payment type and company

"""

'\nSteps to do\n\n1. Get the data from S3\n2. weather data transformations\n3. Do the transformation like in 06 notebooks\n4. Update payment type\n5. Update company\n6. update taxi trips with company and paymant types (replace str values with ids from the latest master tables)\n7. upload the weather to S3\n8. upload the taxi data to S3\n9. upload the newest payment type and company\n\n'

#### Taxi trips transformation code

In [3]:
start_date = (dt.now() - relativedelta(months=2)).strftime("%Y-%m-%d")
url_new = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{start_date}T00:00:00' AND trip_start_timestamp <= '{start_date}T23:59:59'&$limit=213000000"
response_taxi = requests.get(url_new)
taxi_data = response_taxi.json()

In [4]:
taxi_trips = pd.DataFrame(taxi_data)

In [5]:
taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)
taxi_trips.dropna(inplace=True)

In [6]:
taxi_trips.rename(columns={'pickup_community_area' : 'pickup_community_area_id',
                           'dropoff_community_area' : 'dropoff_community_area_id'
                           },inplace=True)

In [8]:
taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('H')
#taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('H')

In [9]:
taxi_trips.head()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,a3584957f6049a08ebd1c24184a16e6e757ef7aa,d899402f96aeec3adabbee95f3a99aba9ade0e0b5c1d93...,2024-03-09T23:45:00.000,2024-03-09T23:45:00.000,432,1.4,32,24,7,0,0,1,8,Cash,City Service,41.878865584,-87.625192142,41.901206994,-87.676355989,2024-03-09 23:00:00
1,ff7d2edf057a5981a008dc03e4e0a408af92c723,bedb270bce40664555c0e645a3f6c242d8f64ed541f6f5...,2024-03-09T23:45:00.000,2024-03-10T00:15:00.000,1721,12.68,76,3,33.75,0,0,6,39.75,Cash,Blue Ribbon Taxi Association,41.980264315,-87.913624596,41.96581197,-87.655878786,2024-03-09 23:00:00
2,fe9e377da9bbd7f69a364c0422fb74a1ec6816a2,141074afe6aae95a4613cde037b5ed73011d4d19017b59...,2024-03-09T23:45:00.000,2024-03-10T00:00:00.000,1260,16.1,76,24,40.25,9.3,0,6,55.55,Credit Card,Taxi Affiliation Services,41.980264315,-87.913624596,41.901206994,-87.676355989,2024-03-09 23:00:00
3,fe8c532732cfa88c8ddbdf7645ce3f6ecee0cdc5,a432fe577b3d7f89c3f91153dab9b75e527e63809a82a3...,2024-03-09T23:45:00.000,2024-03-09T23:45:00.000,414,1.61,7,8,7.25,0,0,0,7.25,Cash,Taxicab Insurance Agency Llc,41.922686284,-87.649488729,41.899602111,-87.633308037,2024-03-09 23:00:00
4,fbeec8a50de88124df91bf14c7f9f743288bb926,16a529a73f019759aa8f1b563ab8511b26a91d65659188...,2024-03-09T23:45:00.000,2024-03-10T00:00:00.000,420,0,32,32,6.5,0,0,2,8.5,Cash,Taxi Affiliation Services,41.880994471,-87.632746489,41.884987192,-87.620992913,2024-03-09 23:00:00


#### Taxi trip transformation function

In [ ]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """
    Transformations for taxi trips DataFrame.

    Parameters:
    - taxi_trips (pd.DataFrame): DataFrame containing taxi trips data.

    Returns:
    - pd.DataFrame: Transformed DataFrame with specified columns dropped, NaN values removed,
      columns renamed, and datetime rounded to the nearest hour for weather matching.
    """
    taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract','pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)
    taxi_trips.dropna(inplace=True)
    taxi_trips.rename(columns={'pickup_community_area' : 'pickup_community_area_id',
                            'dropoff_community_area' : 'dropoff_community_area_id'
                            },inplace=True)
    taxi_trips['datetime_for_weather'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('H')
    return taxi_trips